# Package Auto Assembler

This tool is meant to streamline creation of single module packages.
Its purpose is to automate as many aspects of python package creation as possible,
to shorten a development cycle of reusable components, maintain certain standard of quality
for reusable code. It provides tool to simplify the process of package creatrion
to a point that it can be triggered automatically within ci/cd pipelines,
with minimal preparations and requirements for new modules.


In [1]:
import sys
sys.path.append('../')
from python_modules.package_auto_assembler import (VersionHandler, \
    ImportMappingHandler, RequirementsHandler, MetadataHandler, \
        LocalDependaciesHandler, SetupDirHandler, PackageAutoAssembler)

## Usage examples

The examples contain: 
1. package versioning
2. import mapping
3. extracting and merging requirements
4. preparing metadata
5. merging local dependacies into single module
6. assembling setup directory
7. making a package

### 1. Package versioning

#### Initialize VersionHandler

In [2]:
pv = VersionHandler(
    # required
    versions_filepath = '../tests/package_auto_assembler/lsts_package_versions.yml',
    log_filepath = '../tests/package_auto_assembler/version_logs.csv',
    # optional 
    default_version = "0.0.1")

#### Add new package

In [3]:
pv.add_package(
    package_name = "new_package", 
    # optional
    version = "0.0.1"
    )

#### Update package version

In [4]:
pv.increment_patch(
    package_name = "new_package"
)
## for not tracked package
pv.increment_patch(
    package_name = "another_new_package",
    # optional
    default_version = "0.0.1"
)

There are no known versions of 'another_new_package', 0.0.1 will be used!


#### Display current versions and logs

In [5]:
pv.get_versions(
    # optional
    versions_filepath = 'lsts_package_versions.yml'
)

{'another_new_package': '0.0.1', 'new_package': '0.0.2'}

In [6]:
pv.get_version(
    package_name='new_package'
)

'0.0.2'

In [7]:
pv.get_logs(
    # optional
    log_filepath = 'version_logs.csv'
)

,Timestamp,Package,Version
0,2024-01-01 21:09:37,new_package,0.0.1
1,2024-01-01 21:09:39,new_package,0.0.2
2,2024-01-01 21:09:39,another_new_package,0.0.1


#### Flush versions and logs

In [8]:
pv.flush_versions()
pv.flush_logs()

### 2. Import mapping

#### Initialize ImportMappingHandler

In [9]:
im = ImportMappingHandler(
    # required
    mapping_filepath = "../env_spec/package_mapping.json"
)

#### Load package mappings

In [10]:
im.load_package_mappings(
    # optional
    mapping_filepath = "../env_spec/package_mapping.json"
)

{'PIL': 'Pillow',
 'bs4': 'beautifulsoup4',
 'fitz': 'PyMuPDF',
 'attr': 'attrs',
 'dotenv': 'python-dotenv',
 'googleapiclient': 'google-api-python-client',
 'sentence_transformers': 'sentence-transformers',
 'flask': 'Flask',
 'stdlib_list': 'stdlib-list',
 'sklearn': 'scikit-learn',
 'yaml': 'pyyaml'}

### 3. Extracting and merging requirements

#### Initialize RequirementsHandler

In [12]:
rh = RequirementsHandler(
    # optional
    module_filepath = "../tests/package_auto_assembler/mock_vector_database.py",
    package_mappings = {'PIL': 'Pillow',
                        'bs4': 'beautifulsoup4',
                        'fitz': 'PyMuPDF',
                        'attr': 'attrs',
                        'dotenv': 'python-dotenv',
                        'googleapiclient': 'google-api-python-client',
                        'sentence_transformers': 'sentence-transformers',
                        'flask': 'Flask',
                        'stdlib_list': 'stdlib-list',
                        'sklearn': 'scikit-learn',
                        'yaml': 'pyyaml'},
    requirements_output_path = "../tests/package_auto_assembler/",
    output_requirements_prefix = "requirements_",
    custom_modules_filepath = "../tests/package_auto_assembler/dependancies",
    python_version = '3.8'
)

#### List custom modules for a given directory

In [13]:
rh.list_custom_modules(
    # optional
    custom_modules_filepath="../tests/package_auto_assembler/dependancies")

['comparisonframe', 'shouter']

#### Check if module is a standard python library

In [14]:
rh.is_standard_library(
    # required
    module_name = 'shouter',
    # optional
    python_version = '3.8'
    )

False

#### Extract requirements from the module file

In [15]:
rh.extract_requirements(
    # optional
    module_filepath = "../tests/package_auto_assembler/mock_vector_database.py",
    custom_modules = ['comparisonframe', 'shouter'],
    package_mappings = {'PIL': 'Pillow',
                        'bs4': 'beautifulsoup4',
                        'fitz': 'PyMuPDF',
                        'attr': 'attrs',
                        'dotenv': 'python-dotenv',
                        'googleapiclient': 'google-api-python-client',
                        'sentence_transformers': 'sentence-transformers',
                        'flask': 'Flask',
                        'stdlib_list': 'stdlib-list',
                        'sklearn': 'scikit-learn',
                        'yaml': 'pyyaml'},
    python_version = '3.8'
)

['### mock_vector_database.py',
 'numpy',
 'dill==0.3.7',
 'attrs>=22.2.0',
 'requests==2.31.0',
 'hnswlib==0.7.0',
 'sentence-transformers==2.2.2']

#### Save requirements to a file

In [16]:
rh.write_requirements_file(
    # optional
    module_name = 'mock_vector_database',
    requirements = ['### mock_vector_database.py',
                    'numpy',
                    'dill==0.3.7',
                    'attrs>=22.2.0',
                    'requests==2.31.0',
                    'hnswlib==0.7.0',
                    'sentence-transformers==2.2.2'],
    output_path = "../tests/package_auto_assembler/",
    prefix = "requirements_"
)

#### Read requirements

In [17]:
rh.read_requirements_file(
    # required
    requirements_filepath = "../tests/package_auto_assembler/requirements_mock_vector_database.txt"
)

['numpy',
 'dill==0.3.7',
 'attrs>=22.2.0',
 'requests==2.31.0',
 'hnswlib==0.7.0',
 'sentence-transformers==2.2.2']